In [1]:
print(8)

8


In [2]:
print(8

SyntaxError: incomplete input (891856097.py, line 1)

# AMÉLIORER LE PRODUIT IA DE VOTRE START-UP

<img src="logoAvisRestau.png" alt="logoAvisRestau" width="300" class="center"/>

# SYNTHESE GRAPHIQUE DES RESULTATS

L'objectif ici est de **synthétiser** les principaux résultats de l'analyse :
- des **commentaires négatifs** pour détecter les **sujets d'insatisfaction**
- des **photos** pour détecter leur **catégorie**

(Pour plus de détail et pour parcourir les différentes étapes ayant permis d'atteindre ces résultats, se référer au notebook principal)

In [ ]:
# imports
# custom functions
import myFunctions as mf
%load_ext autoreload

%autoreload 2

# LDA
from gensim.models.ldamodel import LdaModel

# pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# file and directory management
import os
# to make saves
from joblib import dump, load  

# data viz
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
# image annotations
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.colors import ListedColormap, to_rgb

# PARTIE I - ANALYSER LES COMMENTAIRES NÉGATIFS POUR DÉTECTER LES DIFFÉRENTS SUJETS D’INSATISFACTION

## NLP - Méthode

L'objectif **final** est de détecter les sujets de mécontentement dans les avis qui seront postés sur **Avis Restau**. Le but ici n'est pas de réaliser ce projet entièrement **mais de tester sa faisabilité**. Pour cela nous avons la chance de disposer d'un jeu de données existant disponible sur la plateforme **Yelp**. 

Nous réaliserons donc notre études préliminaire sur ce dataset :
- Nous réaliserons les étapes préalables à l'entraînement d'une algorithme de détection de sujets :
    - sélectionner quelques milliers de **commentaires négatifs**
    - **pré-traiter** ces données :
        - mettre en minuscule,
        - tokeniser,
        - normaliser les mots via une lemmatisation,
        - etc.
    - création du **dictionnaire** et représentation de nos reviews en vecteurs **bag-of-words**
    - adapter la **pondération** des mots grâce à un **TF-IDF**
- Nous testerons alors la faisabilité sur ce petit échantillons :
    - utiliser une **technique de réduction de dimension** de type **topic-modeling** pour extraire nos sujets d'insatisfaction sous-jacents de notre corpus. Nous utiliserons ici le `Latent Dirichlet Allocation (LDA)`
    - évaluer notre réduction afin de choisir le bon hyperparamètre `num_topics`
- Enfin nous tâcherons de visualiser nos topics afin d'en détecter les mots-clés. Pour cela nous utiliserons 2 librairies :
    - `wordcloud`
    - `pyLDAvis`

## NLP - Nos commentaires et leur nettoyage

Étapes du preprocessing :
- **charger** un **échantillon** de reviews
- mettre en **minuscules**
- **supprimer** les **url**
- **supprimer** les **séquences d'échappement**
- **corriger** les mots avec des **caractères répétés**
- cleaning général :
    - **supprimer** les **ponctuations**
    - **supprimer** les **stopwords**
    - **supprimer** les **stopwords spécifiques** métier
    - **supprimer** les **nombres** et les nombres écrits en lettres
    - **filtrer** sur les **POS** (en ne gardant que les adjectifs et noms)
    - **normaliser** les mots (ici : **lemmatisation**)

In [ ]:
# load existing .joblib
rev = load("mySaves/rev/rev.joblib")
rev

## NLP - Dictionnaire, BOW et TF-IDF

Nota : Par rapport à l'exemple ci-dessus, nous avons intégré **plus de commentaires : 60000**.

Étapes de traitement :
- création **dictionnaire**
- filtre sur la **fréquence** pour les mots rare et et pour les mots fréquents
- conversion des documents en **bag-of-words**
- pondération des documents par **TF-IDF**

In [ ]:
# load existing .joblib
tfidf_vector = load("mySaves/nlpDictAndVector/tfidf_vector.joblib")
dictionary = load("mySaves/nlpDictAndVector/dictionary.joblib")

Exemple de document après le traitement :

In [ ]:
display(tfidf_vector[16])

## NLP - Latent Dirichlet Allocation et visualisations

**Choix du nombre de topics** basé sur le **score de cohérence** :

In [ ]:
if "coherencesPlot.joblib" in os.listdir("mySaves/coherencesPlot") :
    # load existing .joblib
    load("mySaves/coherencesPlot/coherencesPlot.joblib")

Il n'y a **pas vraiment de pic** ... Nous observons néanmoins **une chute** de cohérence **après 5 topics**. Nous allons regarder ce que cela donne pour **3, 4 et 5 topics** :


### NLP - LDA - 3 topics

In [ ]:
# create the LDA model 
model = LdaModel(
    corpus=tfidf_vector,
    num_topics=3,
    id2word=dictionary,
    random_state=16
)

Vue `wordcloud` et score de cohérence :

In [ ]:
# use custom function wordCloudAndCoherence to display 1 wordcloud for each topic and compute coherence_score
mf.wordCloudAndCoherence(model=model, corpus=tfidf_vector)

Les limites d'une telle visualisation :
- ne nous permet **pas vraiment de comprendre le degré de différence entre les topics**
- apporte du **biais aléatoire dans l'interprétation de l'importance des mots** (orientation, position, couleur)
- ne nous permet pas de visualiser l'**importance relative des topics**

Voyons ce que cela donne avec `PyLDAvis` :

`pyLDAvis` est une librairie spécialisée permettant de :
-  avoir une vue d'ensemble d'un modèle de topic modeling
-  explorer en détail chaque topic
-  explorer en détail chaque mot par rapport aux topics

Il est composé de plusieurs parties :
- ***Intertopic Distance Map*** :
    - représentation 2D **des topics dans l'espace des probabilités de mots dans les topics (matrice topics/mots issue du modèle de LDA)**. Cette **réduction** de dimension 2D est issue d'un **MDS** (multidimensional scaling)
    - la représentation de chaque topic dans cet espace réduit **permet ainsi d'apprécier les distances qui les séparent les uns des autres** dans l'espace plus large, et donc **s'ils sont bien différents**
    - la taille de chaque topic est elle liée à l'**espace des probabilités des topics dans les documents (matrice documents/topics issue du modèle LDA)**. La **somme des probabilités** sur tous les documents de chaque topic permet de définir sa taille. Elle représente donc la **prédominance du topic dans le corpus**
- liste des mots les plus importants :
    - **sans spécifier** un topic : ***Top-30 Most Salient Terms*** :
        - mots classés selon la *saliency* : liée à la fréquence globale de chaque mot dans le corpus.
        - **barres bleus** : fréquences **à l'échelle du corpus**
    - **en sélectionnant** un topic : ***Top-30 Most Relevant Terms for Topic n*** :
        - mots classés selon la *relevance* : liée à la probabilité de chaque mot dans un topic particulier. Ce classement évolue en fonction du paramètre λ :
            - **λ tend vers 1** : la *relevance* met plus en avant **les mots les plus présents au sein du topic**
            - **λ tend vers 0** : la *relevance* normalise cette probabilité du mot pour ce topic avec la probabilité du mot au global dans le corpus. On voit alors apparaître **non pas les mots forcément les plus présents, mais les mots les plus distinctifs du topic analysé**
        - **barres rouges** : fréquences estimées **au sein du topic**
    - il est **aussi possible de sélectionner un mot** en particulier : les **tailles** des topics de l'**Intertopic Distance Map** représentent alors **seulement la répartion de ce mot choisi aux sein des topics**.

In [ ]:
# use the "prepare" function 
vis_data  = gensimvis.prepare(
    topic_model=model,
    corpus=tfidf_vector,
    dictionary=dictionary
)
# display
pyLDAvis.display(vis_data)

Les **3 sujets** de mécontentement :
- le **comportement du personnel et le temps d'attente**, avec des mots comme *table, server, minute, hour, rude, manager, experience, incorrect, customer, hostess, staff*, etc.
- le **rapport qualité / prix**, avec des mots comme *flavor, bland, dry, taste, price, portion, small*, etc.
- la **livraison**, avec des mots comme *delivery, order, hour, service, slow, min, minute, driver, phone, refund*, etc.

### NLP - LDA - 4 topics

In [ ]:
# create the LDA model 
model = LdaModel(
    corpus=tfidf_vector,
    num_topics=4,
    id2word=dictionary,
    random_state=16
)

Vue `wordcloud` et score de cohérence :

In [ ]:
# use custom function wordCloudAndCoherence to display 1 wordcloud for each topic and compute coherence_score
mf.wordCloudAndCoherence(model=model, corpus=tfidf_vector)

Vue `PyLDAvis` :

In [ ]:
# use the "prepare" function 
vis_data  = gensimvis.prepare(
    topic_model=model,
    corpus=tfidf_vector,
    dictionary=dictionary
)
# display
pyLDAvis.display(vis_data)

les **4 sujets** se distinguent un peu plus :
- le **comportement du personnel et le temps d'attente**, avec des mots comme *minute, reservation, order, manager, rude, hour, employee, server, waitress, hostess, service, time, busy, attitude*, etc.
- la **qualité du plat et son prix**, avec des mots comme *flavor, dry, bland, salty, quality, taste, poisonning, soggy, price, portion, small*, etc.
- l'**hygiène**, avec des mots comme *dirty, flour, filthy, clean, hair, glove, mask, fly, bathroom, cleanliness, cockroach, rat, restroom, toilet, disgusting, bug*, etc.
- la **livraison**, avec des mots comme *driver, delivery, uber, masked, order, drive, time, hour, slow, cold*,etc. 

### NLP - LDA - 5 topics

In [ ]:
# create the LDA model 
model = LdaModel(
    corpus=tfidf_vector,
    num_topics=5,
    id2word=dictionary,
    random_state=16
)

Vue `wordcloud` et score de cohérence :

In [ ]:
# use custom function wordCloudAndCoherence to display 1 wordcloud for each topic and compute coherence_score
mf.wordCloudAndCoherence(model=model, corpus=tfidf_vector)

Vue `PyLDAvis` :

In [ ]:
# use the "prepare" function 
vis_data  = gensimvis.prepare(
    topic_model=model,
    corpus=tfidf_vector,
    dictionary=dictionary
)
# display
pyLDAvis.display(vis_data)

Les **5 sujets** de mécontentement :
- l'**ambiance** avec des mots comme *table, reservation, music, loud, party, birthday, noisy, dance*, etc.
- l'**attitude du personnel et le temps d'attente**, avec des mots comme *table, delivery, understaffed, reservation, minute, order, manager, rude, hour, employee, waitress, service, time, busy, attitude*, etc.
- **2 sujets très proches sur la nourriture**, avec des mots en commun comme *flavor, soggy, dry, taste*, etc. :
    -  l'un un peu plus axé sur la **qualité du plat et son prix**, avec des mots comme *portion, price, small, quality, taste*, etc.
    -  l'autre se concentrant plus exclusivement sur la nourriture
- l'**hygiène**, avec des mots comme *dirty, filthy, bathroom, glove, floor, mask, health, restroom, toilet, disgusting, rat, cleaning, roach, hair*, etc.

## NLP - Conclusion

Il est **difficile de conclure sur le nombre idéal de topics** :
- se contenter de 3 met de côté l'hygiène
- conserver 4 topics est intéressant car les sujets sont clairs, avec l'hygiène qui se rajoute,
- enfin avec 5 topics nous avons réussi à obtenir un sujet différent avec l'ambiance, mais la livraison disparaît et le rapport qualité/prix se divise en 2 sous-thèmes pas très clairs...

Nous pouvons néanmoins **conclure sur le faisabilité du projet** :
- le fait d'**ajouter plus de commentaires et d'affiner la préparation des données** a permis de **trouver de vrais sujets de mécontentement**,
- il est donc tout à fait possible :
    - que l'utilisation de **toute la base de données** nous permette de faire disparaître les limitations que nous avons observé
    - qu'une **recherche plus poussée des meilleures paramètres** de notre pipeline (`gridsearchCV`, `BayesSearchCV`, etc.) permette d'améliorer le modèle:
        - filtrage,
        - liste de stopwords,
        - le nombre de topics,
        - les autres hyper-paramètres du modèle (`alpha` et `eta` notamment, qui conditionne le côté plus ou moins *lisse* des distributions de chaque document en topics et de chaque topic en mots)
- **cependant** il faut noter que :
    - nous avons dû ajouter **"à la main"** beaucoup de *stopwords* spécifiques... A voir si ce procédé peut d'une quelconque manière être évité...
    - après quelques essais il s'avère que les **thèmes détectés par le modèle sont instables** (notamment en fonction de la liste des *stopwords* spécifiques)

Il serait intéressant, dans le cadre de la poursuite du projet, de tester d'autres modèles (`LSA`? `BERTopic`? autre ?).

# PARTIE II.A - ANALYSER LES PHOTOS POUR DÉTERMINER LES CATÉGORIES DES PHOTOS - SIFT

## CV-SIFT - Méthode

Nous avons la chance que les photos Yelp soient **déjà labellisées** : L'objectif **final** est d'entraîner un algorithme de **classification supervisée** sur celles-ci afin d'utiliser le modèle ultérieurement **sur les photos qui seront publiées sur Avis Restau**.

Le but ici n'est pas de réaliser ce projet entièrement. L'objectif **préliminaire** est d'**analyser** les photos pour vérifier la faisabilité : 
- Nous réaliserons donc toutes les étapes préalables à l'entraînement d'un algorithme de classification :
    - réaliser le **pré-traitement** sur N images
    - **extraire les features** (nous utiliserons le **SIFT**) pour obtenir les descripteurs de nos N images
    - création des bag-of-visual-words :
        - **trouver les "visual words" grâce à un algorithme KMeans** appliqué sur l'ensemble des descripteurs trouvés
        - construire l'**histogramme de chaque image** en fonction de ce dictionnaire de "visual-words"
        - contruire ainsi notre **matrice des "bag-of-visual-words"**, de taille N images X taille dictionnaire
    - adapter la **pondération** des visual-words grâce à un **TF-IDF**<br>
    
    **... mais au lieu de réaliser ce travail sur un très grand nombre de photos (avec un split préalable en train/test set, etc.), nous ne prendrons que quelques photos de chaque catégorie.**<br>

- Nous testerons alors la **faisabilité sur ce petit échantillon** avec un **clustering KMeans** avec **K = le nombre de labels** pour vérifier que nos features peuvent bien regrouper nos labels :
    - réduction de **dimension préalable** via une ACP
    - appliquer le KMeans
    - effectuer une mesure de **similarité ARI**
    - analyser les catégories les mieux regroupées (via une **matrice de confusion** entre les catégories et les clusters prédits)<br><br>


- Nous tâcherons également de visualiser nos N images dans cet espace des features pour **analyser si les catégories/labels fournies ressortent** :
    - utilisation du même espace réduit par l'ACP
    - réduction de dimension par **T-SNE**, idéal pour représenter des données en grande dimension
    - visualisation :
        - des catégories
        - des clusters

## CV-SIFT - Nos image et leur traitement

Étapes du preprocessing :
- **charger** un **échantillon** d'images
- **mettre** en nuance de **gris**
- **égalisation** d'**histogramme**
- **filtre gaussien**
- **extraire** les features **descripteurs SIFT**

Nous pouvons regarder ce que cela donne visuellement :

In [ ]:
# load existing .joblib
examplesPhotoDict = load("mySaves/examplesPhotoDict/examplesPhotoDict.joblib")

# create a figure
fig, axs = plt.subplots(2,5, figsize=(14,6))

# choose a category example
cat = "drink"

# plot the two images stored in the dict for this category, for each preprocessing step
for i,phase in enumerate(['raw', 'gray', 'equal', 'gaussian', 'keypoints']) :
    for j,img in enumerate(examplesPhotoDict[phase][cat]) :
        # plot
        # handle grayscale
        if len(img.shape) == 2 :
            axs[j,i].imshow(img, cmap='gray',vmin=0,vmax=255)
        else :
            axs[j,i].imshow(img)

        # set anchor
        axs[j,i].set_anchor("N")
        # remove axis
        axs[j,i].axis(False)
        # title on top axes 
        if j==0 :
            axs[j,i].set_title(phase)

# sup title
fig.suptitle("Photo cleaning steps")

plt.show()

## CV-SIFT - Dictionnaire, Bag-of-Visual-Words, TF-IDF, PCA

Étapes de traitement :
- Création d'un **dictionnaire de visual words** :
    - rassemblement de tous les descripteurs des images
    - utilisation d'un algorithme de clustering (`MiniBatchKMeans`) : chaque centroïde de cluster correspond à un *visual word*
- Création **matrice des Bag-of-Visual-Words** :
    - pour chaque descripteur de chaque image, appliquer le modèle de clustering (dictionnaire) pour attibuer le visual word correspondant
    - pour chaque image, compter le nombre d'occurences de chaque *visual word* présent
    - intégrer ces histogrammes à une matrice (nombre d'images X nombre de *visual words*)
- Appliquer la pondération **TF-IDF**
- Appliquer une analyse en composantes principales **PCA** pour :
    - réduire le nombre de dimension (*curse of dimensionality*)
    - faciliter le travail du futur modèle (temps et mémoire)

Nous avons alors un dataframe comportant une ligne par image et une colonne par composante principale :

In [ ]:
# load existing .joblib
siftFeatures = load("mySaves/siftFeatures/siftFeatures.joblib")
siftFeatures

## CV-SIFT - Clustering - Kmeans

Nous allons maintenant vérifier s'il serait **faisable** de labelliser automatiquement nos photos. Pour cela nous n'allons pas utiliser d'algorithme de classification supervisée (pour rappel nous avons fait le choix d'utiliser peu de données), mais un **modèle de clustering**. Cela nous permettra d' **analyser, à ce stade préliminaire, si les features que nous venons de créer permettent de regrouper nos catégories dans des clusters**.

Étapes de l'étude de faisabilité :
- **algorithme de clustering** avec, pour rappel, **K = le nombre catégories de photo**
- calcul de l'**ARI** entre les labels des clusters et les catégories
- **projection** de nos données **en 2D** grâce à un **TSNE**
- **représentation** des clusters et des vraies catégories **sur cet espace**


L'**ARI** pour cette méthode SIFT n'est pas très bon : **< 0.15 ...** Notre clustering n'est pas une réussite. On le remarque assez bien sur la projection T-SNE :

In [ ]:
# use existing .joblib
load("mySaves/figures/figSIFTscatter.joblib")

La seule classe intéressante en termes de résultat est **"menu"**. On peut vérifier également cela en remplaçant quelques points par leurs images respectives :

<img src="mySaves/figures/figSIFTphotos.png" alt="figSIFTphotos" class="center"/>

Le **SIFT ne nous permet pas vraiment de conclure positivement** quant à l'opportunité de développer un modèle de classification supervisée... Essayons maintenant avec du Transfer Learning :

# PARTIE II.B - ANALYSER LES PHOTOS POUR DÉTERMINER LES CATÉGORIES DES PHOTOS - TRANSFER LEARNING

## CV-TL - Utiliser un VGG16 pré-entraîné comme un extracteur de features autonome

Ici les étapes de traitement diffèrent un peu : 

- Charger nos photos et **les préparer** pour le `VGG16` :
    - les charger à la taille **224 x 224**
    - les convertir en **array**
    - les **rassembler** en un seul array
    - les convertir en **BGR**
    - **centrer** chaque canal de couleur
- Utiliser un `VGG16` **déjà entraîné** comme un **extracteur de features** :
    - **retirer la dernière couche fully-connected** (celle contenant la fonction d'activation Softmax nous donnant normalement nos scores par classe)
    - **se servir de la sortie de la couche précédente** comme **features**
- Appliquer également une réduction de dimensions par `PCA`

In [ ]:
# load existing .joblib
vgg16Features = load("mySaves/transferLearningFeatures/reducedVGG16featuresDf.joblib")
vgg16Features

## CV-TL - Clustering - Kmeans

L'**ARI** pour cette méthode Transfer Learning est bien meilleur : **> 0.70 ...** On le remarque assez bien sur la projection T-SNE :

In [ ]:
# use existing .joblib
load("mySaves/figures/figTLscatter.joblib")

La **correspondance est bonne**. On peut vérifier également cela en remplaçant quelques points par leurs images respectives :

<img src="mySaves/figures/figTLphotos.png" alt="figTLphotos" class="center"/>

**Le VGG16 en tant qu'extracteur de features donne de bien meilleurs résultats.**

On peut **conclure** qu'il serait **opportun de poursuivre le projet** avec le **développement d'un modèle de classification supervisée** (avec bien plus de photos).